## Строим вопрос-ответного бота по технологии Retrieval-Augmented Generation на LangChain

[LangChain](https://python.langchain.com) - это набирающая популярность библиотека для работы с большими языковыми моделями и для построения конвейеров обработки текстовых данных. В одной библиотеке присутствуют все элементы, которые помогут нам создать вопрос-ответного бота на наборе текстовых данных: вычисление эмбеддингов, запуск больших языковых моделей для генерации текста, поиск по ключу в векторных базах данных и т.д.

Для начала, установим `langchain` и сопутствующие библиотеки.

In [2]:
%pip install langchain sentence_transformers lancedb unstructured

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/89/b2/3b74b85356662637bfe3efbc6462ccb28227215fcf8e07b5e9a830f5c661/langchain-0.0.275-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for lancedb from https://files.pythonhosted.org/packages/0b/59/8800508af2c0afe0269278432ed5a7a8b03bebc75826ac211ee4053bf7a1/lancedb-0.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for unstructured from https://files.pythonhosted.org/packages/e3/97/478e5f01e8922acc140ee35adde4c5e6861f7a693fe57daafe53947e6602/unstructured-0.10.8-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json<0.6.0,>=0.5.7 from https://files.pythonhosted.org/packages/97/5f/e7cc90f36152810cab08b6c9c1125e8bcb9d76f8b3018d101b5f877b386c/datac

## Разбиваем документ на части

Для работы retrival augmented generation нам необходимо по запросу найти наиболее релевантные фрагменты исходного текста, на основе которых будет формироваться ответ. Для этого нам надо разбить текст на такие фрагменты, по которым мы сможем вычислять эмбеддинг, и которые будут с запасом помещаться во входное окно используемой большой языковой модели.

Для этого можно использовать механизмы фреймворка LangChain - например, `RecursiveCharacterTextSplitter`. Он разбивает текст на перекрывающиеся фрагменты по набору типовых разделителей - абзацы, переводы строк, разделители слов.

In [1]:
import langchain
import langchain.document_loaders

source_dir = "/home/jupyter/mnt/s3/mclass/text"

loader = langchain.document_loaders.DirectoryLoader(source_dir,glob="*.txt",show_progress=True,recursive=True)
splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=50)
fragments = splitter.create_documents([ x.page_content for x in loader.load()])
len(fragments)

100%|██████████| 3/3 [00:03<00:00,  1.03s/it]


259

## Вычисляем эмбеддинги для всех фрагментов

Для вычисления эмбеддингов можно взять какую-нибудь модель из репозитория HuggingFace, с поддержкой русского языка. LangChain содержит свои встроенные средства работы с эмбеддингами, и поддерживает модели из HuggingFace: 

In [42]:
embeddings = langchain.embeddings.HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v1")
sample_vec = embeddings.embed_query("Hello, world!")
len(sample_vec)

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


512

In [43]:
import requests

iam_token = "t1.9euelZqUlZjJm5qRyJjOzJ2WmZ7OnO3rnpWalpOXl5iQzJqTyZeVzJaRl5vl8_dJTDlY-e8jFWFG_N3z9wl7Nlj57yMVYUb8zef1656VmpGUzIydyZHJyM3GjIqYnMqM7_zF656VmpGUzIydyZHJyM3GjIqYnMqM.BkHA7mr54HYTHGyb6frKXXGIfLuQOYB3S-yevqHpZzpp2CeKPavBaYGUFiKfNV-fNxfvxGoghwBfOUIU-NjdDw"
folder_id="b1gbicod0scglhd49qs0"
headers={ 
    "Authorization" : f"Bearer {iam_token}",
    "x-folder-id" : folder_id
}
j = {
  "model" : "general",
  "text": "Hello, world!"
}

res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/embedding",json=j,headers=headers)
res

<Response [404]>

## Cохраняем эмбеддинги  в векторную БД

Для поиска нам нужно уметь быстро сопоставлять эмбеддинг запроса, и эмбеддинги всех фрагементов наших исходных материалов. Для этого используются векторные базы данных 

In [3]:
from langchain.vectorstores import LanceDB
import lancedb

db_dir = "../store"

db = lancedb.connect(db_dir)
table = db.create_table(
    "vector_index",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

Failed to deserialize variable 'db'. Run the following code to delete it:
  del_datasphere_variables('db', 'embeddings', 'retriever', 'table')
Traceback (most recent call last):
  File "/kernel/lib/python3.10/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/home/jupyter/.local/lib/python3.10/site-packages/lance/dataset.py", line 71, in __init__
    self._ds = _Dataset(
ValueError: Dataset at path home/jupyter/work/resources/store/vector_index.lance/_versions/2.manifest was not found: Not found: home/jupyter/work/resources/store/vector_index.lance/_versions/2.manifest

The above exception was the direct cause of the following exception:

ml_kernel.state.state_protocol.KernelStateProtocol.DeserializationException: ['db', 'embeddings', 'retriever', 'table']
Failed to deserialize variable 'db'. Run the following code to delete it:
  del_datasphere_variables('db', 'embeddings', 'retriever', 'table')
Traceback (most recent c

In [4]:
db = LanceDB.from_documents(fragments, embeddings, connection=table)

In [6]:
q="Почему стоит работать в Яндексе?"

res = db.similarity_search(q)
for x in res:
    print('-'*40)
    print(x.page_content)

----------------------------------------
и работать да какая проблема о том что у нас есть смотри ка для русского языка захотели мы сделать казахский язык захотели сделать английский язык захотели сделать еще 5 языков И в этот момент каждый такой язык стоит дорого потому что им надо знать людям потому что это проектная работа становится и по железу то есть мы начинаем держать несколько разных инсталляций каждый из которых требует же плюшек при этом всем известно как это работает трафик на новых сервисах у вас есть так вот так вот большой трафик и
----------------------------------------
на криптоинтах так далее организовать это в зуме достаточно сложно клинической среде постоянно есть конференции на которой ученые ездят обсуждают какие то идеи вот поэтому У нас есть возможность там как то работать удаленно но это всегда кейс бэкейс В зависимости от человека вот пока что показывает что Люди которые давно работают с большим опытом они в принципе могут работать удаленно но ожидать от напр

In [7]:
retriever = db.as_retriever(
    search_kwargs={"k": 5}
)
res = retriever.get_relevant_documents(q)

In [8]:
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
import requests

class YandexLLM(langchain.llms.base.LLM):
    iam_token: str
    folder_id: str
    max_tokens : int = 1500
    temperature : float = 1
    instruction_text : str = None

    @property
    def _llm_type(self) -> str:
        return "yagpt"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        req = {
          "model": "general",
          "instruction_text": self.instruction_text,
          "request_text": prompt,
          "generation_options": {
            "max_tokens": self.max_tokens,
            "temperature": self.temperature
          }
        }
        res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/instruct",
          headers=
            { "Authorization" : f"Bearer {self.iam_token}",
              "x-folder-id" : self.folder_id }, json=req).json()
        return res['result']['alternatives'][0]['text']

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"max_tokens": self.max_tokens, "temperature" : self.temperature }

In [11]:
iam = "t1.9euelZrIzZKXj5iUy82OlJWezpOQle3rnpWalpOXl5iQzJqTyZeVzJaRl5vl9PcabkZY-e80fUvD3fT3WhxEWPnvNH1Lw83n9euelZqOisfOyI2QkJuemJLGjZvOyu_8xeuelZqOisfOyI2QkJuemJLGjZvOyg.OTDSozh2X04QzrfbvcVxnjBBkxgrRQ_GIzY4TKfWEEut2VumQN1qBuyTWAl8REVdALeooZd_qa8iroTSB6jKDA"

instructions = """
Представь себе, что ты сотрудник Yandex Cloud. Твоя задача - подробно отвечать на все вопросы собеседника."""

llm = YandexLLM(iam_token=iam, folder_id="b1gbicod0scglhd49qs0",
                instruction_text = instructions)

In [12]:
llm(q)

'Приветствую! Как сотрудник Yandex Cloud я действительно понимаю, насколько важно убедить людей в том, почему работать в Яндексе это блестящая идея. Вот несколько причин, по которым работа здесь может быть необыкновенно захватывающей:\n\n- Сотрудничество с одними из лучших в мире специалистов: Yandex является крупнейшей высокотехнологичной компанией в России, и мы сотрудничаем с наиболее опытными коллегами в своей отрасли. Наша команда зависит от профессионализма каждого отдельного члена команды, который развивается и улучшает свои навыки на ежедневной основе.\n\n- Инновации на каждом шагу: Все без исключения команды Яндекса работают над инновационными проектами, цель которых – развивать новые, захватывающие продукты и технологии, которые делают жизнь людей лучше. Специализированные подразделения Яндекса занимаются разработкой интеллектуальных интерфейсов и инструментов для нашей платформы хранения данных – Yandex Database. "yandex/db" – наша операционная система для баз данных, котора

In [13]:
# We prepare and run a custom Stuff chain with reordered docs as context.

# Override prompts
document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
stuff_prompt_override = """Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
Текст:
-----
{context}
-----
Вопрос:
{query}"""
prompt = langchain.prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Instantiate the chain
llm_chain = langchain.chains.LLMChain(llm=llm, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
chain.run(input_documents=res, query=q)

'Во-первых, Яндекс - это мировая компания с большими возможностями для развития и роста в профессиональном плане. В Яндекс работают известные мировые лидеры, которые создают инновационные продукты, преобразовывающие сферы человеческих знаний и деятельности. Здесь ты сможешь раскрыть свой потенциал и развить новые навыки, а также получить корпоративные знания и опыт.\n\nВо-вторых, в Яндексе ценят ответственный подход к работе и высокую степень профессионализма, создают все условия для эффективной работы и обучения сотрудников, которые готовы прийти на помощь в любых ситуациях.\n\nВ-третьих, благодаря облачным услугам и сервисам Яндекс, его клиенты по всему миру могут высоко оценить качество и возможности развития своих компаний, предоставляя бизнесу и пользователям продукта многие из самых сложных и инновационных решений в Интернете. Каждый день лично и во взаимодействии со своими коллегами ты становишься частью больших и масштабных проектов, участники и авторы которых работают над клие

In [15]:
from langchain.document_transformers import LongContextReorder
reorderer = LongContextReorder()

def answer(query,reorder=True):
  results = retriever.get_relevant_documents(query)
  if reorder:
    results = reorderer.transform_documents(results)
  return chain.run(input_documents=results, query=query)

In [17]:
answer("Что ты можешь сказать об ML-команде Яндекс-облака?")

'Да, такая команда существует. Она отвечает за разработку и внедрение алгоритмов и машинному обучению. Они предоставляют гибкие решения для разработки, тестирования, развертывания и использования моделей машинного обучения. Команда ML также занимается совместной работой с другими командами продукта, такими как DevOps и аналитики данных.В ее обязанности входит обучение моделей данных.'

In [18]:
def compare(q):
    print(f"Ответ YaGPT: {llm(q)}")
    print(f"Ответ бота: {answer(q)}")
    
compare("Что ты можешь сказать об ML-команде Яндекс-облака?")

Ответ YaGPT: Конечно, стоит вопрос и я смогу помочь на него ответить и предоставлю всю возможную необходимую информацию:

ML-команда Yandex.Cloud занимается разработкой технологий машинного обучения для всех облачных продуктов компании. Команда создает и совершенствует черные ящики, которые могут предсказывать новые сегменты данных и способны перестраиваться для новых запросов и моделей бизнеса
Ответ бота: Команда Яндекс  Облака постоянно использует ML (машинное  обучение), совершенствуя продукты по мере востребованности. Сотрудники активно участвуют в исследуемых темах, имеют отличную квалификацию и способности решать сложные задачи.

За прошедший год команда Яндекса запустила образовательные ресурсы нового уровня, задала новый стандарт презентации результатов научным исследованиям. Теперь результаты этих исследований доступны миллионам открытым доступом по решению Европейской комиссии, а также случайно образованной публичной группе специалистов в области машинного обучения.
